In [1]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import array
from scipy.interpolate import RegularGridInterpolator
import sys
import math
from skmisc.loess import loess
from scipy.interpolate import interpn

In [ ]:
def main(day):
    # print('in')
    fn      = '/glade/work/geethma/research/data_cygnss/2019/cyg.ddmi.s201901'+f'{day:02d}'+'-003000-e201901'+f'{day:02d}'+'-233000.l3.grid-wind.a30.d31.nc'
    dsi     = nc.Dataset(fn)
    # ds.append(dsi)
    lons	= dsi.variables['lon'][:]           #size=1800 #Range is 0.1 .. 359.9 #resolution=0.2
    lats	= dsi.variables['lat'][:]        #size=400 #Range is -39.9 .. 39.9 #resolution=0.2
    u		= dsi.variables['wind_speed'][:,:,:]    #float wind_speed(time, lat, lon) #units = "m s-1"
    globals()['time']    = dsi.variables['time'][:]          #size=24
    dsi.close
##############
    fna     = '/glade/work/geethma/research/data_amsr2/2019/RSS_AMSR2_ocean_L3_daily_2019-01-'+f'{day:02d}'+'_v08.2.nc'
    dsa     = nc.Dataset(fna)
    lonsa   = dsa.variables['lon'][:]          #size=1440  #Range is 0.125 .. 359.875 #resolution=0.25
    latsa	= dsa.variables['lat'][:]          #size=720 #Range is -89.875 .. 89.875 #resolution=0.25
    ua      = dsa.variables['wind_speed_MF'][:,:,:]  #float32 wind_speed_LF(pass, lat, lon)
    timea   = dsa.variables['time'][:,:,:]            #(pass, lat, lon) shape=(2,720,1440)
    passa   = dsa.variables['pass'][:]
    # raina   = dsa.variables['rain_rate'][:,:,:]
    dsa.close
# ############
# ##############
    # x = lons
    x = np.array(lons)
    y = np.array(lats)
    globals()['cyg_lat'] = y
    globals()['cyg_lon'] = x
    z = np.array(time)
    zg, yg, xg = np.meshgrid(z,y, x, indexing='ij', sparse=True)

    globals()['ui'] = interpn((z,y,x), u,(zg,yg,xg))

    xa = np.array(lonsa)
    ya = np.array(latsa)
    globals()['ams_lat'] = ya
    globals()['ams_lon'] = xa
    za = np.array(passa)
    zga, yga, xga = np.meshgrid(za, ya, xa, indexing='ij', sparse=True)

    globals()['uai'] = interpn((za,ya,xa), ua, (zga,yga,xga))#a function [1:2,-89.9:89.9,0.1:359]
    globals()['timeai'] = interpn((za,ya,xa), timea, (zga,yga,xga))
    # globals()['my_interpolate_r'] = RegularGridInterpolator((za,ya,xa), raina, method='nearest') #rain_rate
    ##########
    #cyg | lat 4,399 | lon 0,1799
    #amsr | lat 203,519 | lon 0,1439
    print(ui.max(),uai.max())
    print(yg[(0,4,0)],yg[(0,5,0)],yg[(0,6,0)],yg[(0,399,0)])
    print('n',yga[(0,203,0)],yga[(0,204,0)],yga[(0,205,0)],yga[(0,519,0)])
    